In [ ]:
# @title Copyright & License (click to expand)
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# GCP上的E2E ML：MLOps阶段7：监控：使用TensorFlow Serving容器的自定义表格模型的Vertex AI模型监控

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage7/get_started_with_model_monitoring_custom_tf_serving.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage7/get_started_with_model_monitoring_custom_tf_serving.ipynb">
        <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td> 
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/ml_ops/stage7/get_started_with_model_monitoring_custom_tf_serving.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI Workbench中打开
    </a>
  </td>
</table>

## 概述

本教程演示了如何在自定义表格模型和自定义部署容器上使用Vertex AI模型监控。

### 目标

在本笔记本中，您将学习如何使用`Vertex AI模型监控`服务来检测输入预测请求中的特征偏离和漂移，针对自定义表格模型，使用自定义部署容器。在本教程中，将使用TensorFlow Serving作为自定义部署容器。

本教程使用以下Google Cloud ML服务：

- `Vertex AI模型监控`
- `Vertex AI预测`
- `Vertex AI模型`资源
- `Vertex AI端点`资源

执行的步骤包括：

- 下载一个预训练的自定义表格模型。
- 将预训练模型上传为`模型`资源。
- 将`模型`资源部署到带有`TensorFlow Serving`服务二进制文件的`端点`资源。
- 配置`端点`资源进行模型监控。
- 生成用于检测偏离的合成预测请求。
- 等待电子邮件警报通知。
- 生成用于检测漂移的合成预测请求。
- 等待电子邮件警报通知。

了解更多关于[介绍 Vertex AI 模型监控](https://cloud.google.com/vertex-ai/docs/model-monitoring/overview)。

### 模型

本教程使用一个预训练模型，模型工件存储在一个公共的云存储通（Cloud Storage bucket）中。

该模型基于[这篇博客文章](https://cloud.google.com/blog/topics/developers-practitioners/churn-prediction-game-developers-using-google-analytics-4-ga4-and-bigquery-ml)。这个模型的理念是，您的公司有大量的日志数据，描述了您的游戏用户如何与网站互动的信息。原始数据包含以下类别的信息：

- 身份 - 独特的玩家身份号码
- 人口统计特征 - 关于玩家的信息，比如玩家所位于的地理区域
- 行为特征 - 玩家触发特定游戏事件次数的计数，比如达到新的等级
- 流失倾向 - 这是标签或目标特征，它提供了这位玩家可能流失的预估概率，即停止成为一个活跃的玩家。

### 成本

本教程使用Google Cloud的计费组件：

* Vertex AI
* BigQuery
* Cloud Storage

了解[Vertex AI价格](https://cloud.google.com/vertex-ai/pricing)和[Cloud Storage价格](https://cloud.google.com/storage/pricing)，并使用[Pricing Calculator](https://cloud.google.com/products/calculator/) 根据您的预期使用情况生成成本估算。

### 设置本地开发环境

**如果您使用的是Colab或Vertex AI Workbench笔记本**，您的环境已经满足运行此笔记本的所有要求。您可以跳过这一步。

**否则**，请确保您的环境满足这个笔记本的要求。
您需要以下内容：

* Google Cloud SDK
* Git
* Python 3
* virtualenv
* 在使用Python 3的虚拟环境中运行的Jupyter笔记本

Google Cloud的[设置Python开发环境指南](https://cloud.google.com/python/setup)和[Jupyter安装指南](https://jupyter.org/install)提供了满足这些要求的详细说明。以下步骤提供了一套简化的指令：

1. [安装并初始化Cloud SDK。](https://cloud.google.com/sdk/docs/)

1. [安装Python 3。](https://cloud.google.com/python/setup#installing_python)

1. [安装virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv)并创建一个使用Python 3的虚拟环境。激活虚拟环境。

1. 要安装Jupyter，请在终端窗口的命令行中运行 `pip install jupyter`。

1. 要启动Jupyter，请在终端窗口的命令行中运行 `jupyter notebook`。

1. 在Jupyter Notebook仪表板中打开此笔记本。

安装

安装执行此笔记本所需的软件包。

In [ ]:
import os
import sys

assert sys.version_info.major == 3, "This notebook requires Python 3."

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME") and not os.getenv("VIRTUAL_ENV")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

# Install required packages.
! pip3 install {USER_FLAG} --quiet --upgrade google-cloud-aiplatform \
                                             google-cloud-bigquery

### 重新启动内核

安装SDK后，您需要重新启动笔记本内核，以便它可以找到包。您可以从*Kernel -> 重新启动内核*重新启动内核，或者运行以下命令：

In [ ]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

在开始之前

设置您的Google Cloud项目

无论您使用的是哪种笔记本环境，都需要进行以下步骤。

1. [选择或创建一个Google Cloud项目](https://console.cloud.google.com/cloud-resource-manager)。当您首次创建帐户时，您将获得300美元的免费信用额，可用于支付计算/存储成本。

2. [确保为您的项目启用了计费功能](https://cloud.google.com/billing/docs/how-to/modify-project)。

3. [启用Vertex AI API和Compute Engine API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component)。

4. 如果您在本地运行本笔记本，您需要安装[Cloud SDK](https://cloud.google.com/sdk)。

5. 在下面的单元格中输入您的项目ID。然后运行该单元格，以确保Cloud SDK在本笔记本中的所有命令中都使用正确的项目。

**注意**：Jupyter以`!`开头的行作为shell命令运行，并将以`$`开头的Python变量插入这些命令中。

#### 设置您的项目ID

**如果您不知道您的项目ID**，您可以尝试使用`gcloud`来获取您的项目ID。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### 区域

您还可以更改“REGION”变量，该变量用于笔记本其余部分的操作。以下是Vertex AI支持的区域。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太地区：`asia-east1`

**对于此笔记本，我们建议将区域设置为默认值us-central1**。

您不能使用多区域存储桶进行Vertex AI的训练。并非所有区域对所有Vertex AI服务提供支持。

了解有关[Vertex AI区域](https://cloud.google.com/vertex-ai/docs/general/locations)的更多信息。

In [ ]:
REGION = "[your-region]"  # @param {type: "string"}

if REGION == "[your-region]":
    REGION = "us-central1"

UUID

如果您正在进行现场教程，您可能正在使用共享的测试帐户或项目。为了避免在创建的资源之间发生名称冲突，您需要为每个实例会话创建一个uuid，并将其附加到您在本教程中创建的资源的名称上。

In [ ]:
import random
import string


# Generate a uuid of a specifed length(default=8)
def generate_uuid(length: int = 8) -> str:
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))


UUID = generate_uuid()

用户电子邮件

设置您的用户电子邮件地址以接收监控警报。

In [ ]:
USER_EMAIL = "[your-user-email]"  # @param {type:"string"}

###验证您的Google Cloud账户

**如果您正在使用Vertex AI Workbench笔记本**，您的环境已经通过身份验证。

**如果您使用的是Colab**，运行下面的单元格，并按提示进行身份验证。

**否则**，请按照以下步骤操作：

1. 在Cloud Console中，转到[**创建服务帐户密钥**页面](https://console.cloud.google.com/apis/credentials/serviceaccountkey)。

2. 点击**创建服务帐户**。

3. 在**服务帐户名称**字段中输入名称，然后点击**创建**。

4. 在**赋予此服务帐户对项目的访问权限**部分，点击**角色**下拉列表。在过滤框中输入"Vertex AI"，并选择**Vertex AI管理员**。在过滤框中输入"存储对象管理员"，并选择**存储对象管理员**。

5. 点击**创建**。包含您密钥的JSON文件将下载到您的本地环境中。

6. 在下面的单元格中将您的服务帐户密钥路径输入为`GOOGLE_APPLICATION_CREDENTIALS`变量，并运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Vertex AI Workbench, then don't execute this code
IS_COLAB = "google.colab" in sys.modules
if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

关于服务账户和权限的注释

**默认情况下不需要任何配置**，如果遇到任何权限相关问题，请确保上述服务账户具有所需的角色：

|服务账户邮箱|描述|角色|
|---|---|---|
|PROJECT_NUMBER-compute@developer.gserviceaccount.com|Compute Engine默认服务账户|Dataflow管理员、Dataflow Worker、存储管理员、BigQuery管理员、Vertex AI用户|
|service-PROJECT_NUMBER@gcp-sa-aiplatform.iam.gserviceaccount.com|AI 平台服务代理|Vertex AI 服务代理|

1. 进入 https://console.cloud.google.com/iam-admin/iam。
2. 勾选 "包括由 Google 提供的角色授权" 复选框。
3. 查找上述邮箱。
4. 授予相应的角色。

### 使用来自不同项目的数据源
- 对于BQ数据源，请为两个服务账户授予 "BigQuery 数据查看器" 角色。
- 对于CSV数据源，请为两个服务账户授予 "存储对象查看器" 角色。

### 创建 Cloud 存储桶

**无论您的笔记本环境如何，都需要执行以下步骤。**

在下面设置您的 Cloud 存储桶的名称，在本教程中您将使用该存储桶上传监控服务的 `输入模式`。

存储桶名称必须在所有谷歌云项目中全局唯一，包括您组织之外的项目。

In [ ]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "aip-" + UUID
    BUCKET_URI = "gs://" + BUCKET_NAME

只有在您的存储桶不存在时：运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_URI

最后，通过检查您云存储桶的内容来验证对其的访问权限：

In [ ]:
! gsutil ls -al $BUCKET_URI

### 导入库

In [ ]:
import google.cloud.aiplatform as aiplatform
from google.cloud import bigquery
from google.cloud.aiplatform import model_monitoring

### 初始化 Python 的 Vertex AI SDK

为您的项目和相应的存储桶初始化 Python 的 Vertex AI SDK。

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION)

在本教程中，您将使用相同的公共BigQuery表中的数据，该数据用于训练预训练模型。您将创建一个客户端接口，然后使用它来访问数据。

In [ ]:
bqclient = bigquery.Client(project=PROJECT_ID)

设置硬件加速器

您可以为预测设置硬件加速器（例如，GPU），或选择不使用任何（CPU）。硬件加速器降低了对预测请求的延迟响应。在选择硬件加速器时，要考虑延迟与额外成本之间的权衡。

将变量`DEPLOY_GPU/DEPLOY_NGPU`设置为使用支持GPU的容器镜像以及分配给虚拟机实例（VM）的GPU数目。例如，要使用一个GPU容器镜像，并为每个VM分配4个Nvidia Tesla K80 GPU，您可以指定：

（aiplatform.gapic.AcceleratorType.NVIDIA_TESLA_K80, 4）

请查看[可用加速器的位置](https://cloud.google.com/vertex-ai/docs/general/locations#accelerators)。

否则，指定（无，无）以使用能在CPU上运行的容器镜像。

In [ ]:
GPU = False
if GPU:
    DEPLOY_GPU, DEPLOY_NGPU = (aiplatform.gapic.AcceleratorType.NVIDIA_TESLA_K80, 1)
else:
    DEPLOY_GPU, DEPLOY_NGPU = (None, None)

设置预构建的容器

设置用于预测的预构建 Docker 容器镜像。

有关最新列表，请参阅[预构建的容器用于预测](https://cloud.google.com/ai-platform-unified/docs/predictions/pre-built-containers)。

In [ ]:
if GPU:
    DEPLOY_VERSION = "tf2-gpu.2-5"
else:
    DEPLOY_VERSION = "tf2-cpu.2-5"

DEPLOY_IMAGE = "{}-docker.pkg.dev/vertex-ai/prediction/{}:latest".format(
    REGION.split("-")[0], DEPLOY_VERSION
)

print("Deployment:", DEPLOY_IMAGE, DEPLOY_GPU, DEPLOY_NGPU)

#### 设置机器类型

接下来，设置用于训练和预测的机器类型。

- 将变量`DEPLOY_COMPUTE`设置为配置用于预测的计算资源。
 - `机器类型`
     - `n1-standard`：每个vCPU 3.75GB的内存
     - `n1-highmem`：每个vCPU 6.5GB的内存
     - `n1-highcpu`：每个vCPU 0.9GB的内存
 - `vCPUs`：\[2, 4, 8, 16, 32, 64, 96\]台

*注意：您还可以在训练和部署中使用n2和e2机器类型，但它们不支持GPU。*

In [ ]:
MACHINE_TYPE = "n1-standard"

VCPU = "4"
TRAIN_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Train machine type", TRAIN_COMPUTE)

MACHINE_TYPE = "n1-standard"

VCPU = "4"
DEPLOY_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Deploy machine type", DEPLOY_COMPUTE)

### 启用Artifact Registry API

您必须为您的项目启用Artifact Registry API服务。

了解更多关于[启用服务](https://cloud.google.com/artifact-registry/docs/enable-service)。

In [ ]:
! gcloud services enable artifactregistry.googleapis.com

创建一个私有的Docker仓库

第一步是在Google Artifact Registry中创建您自己的Docker仓库。

1. 运行`gcloud artifacts repositories create`命令，在您的区域中创建一个新的Docker仓库，并添加描述为"docker仓库"。

2. 运行`gcloud artifacts repositories list`命令来验证您的仓库是否已经创建。

In [ ]:
PRIVATE_REPO = "my-docker-repo"

! gcloud artifacts repositories create {PRIVATE_REPO} --repository-format=docker --location={REGION} --description="Docker repository"

! gcloud artifacts repositories list

### 配置私有存储库的身份验证

在推送或拉取容器映像之前，配置Docker使用`gcloud`命令行工具对您地区的`Artifact Registry`发起请求进行身份验证。

In [ ]:
! gcloud auth configure-docker {REGION}-docker.pkg.dev --quiet

用于提供服务的容器（Docker）镜像

设置用于提供预测的TensorFlow Serving Docker容器镜像。

1. 从Docker Hub拉取相应的CPU或GPU TF Serving Docker镜像。
2. 为图像创建一个标签，以在Artifact Registry中注册。
3. 使用Artifact Registry注册图像。

了解有关[TensorFlow Serving](https://www.tensorflow.org/tfx/serving/docker)的更多信息。

In [ ]:
# Executes in Vertex AI Workbench
if DEPLOY_GPU:
    DEPLOY_IMAGE = (
        f"{REGION}-docker.pkg.dev/"
        + PROJECT_ID
        + f"/{PRIVATE_REPO}"
        + "/tf_serving:gpu"
    )
    TF_IMAGE = "tensorflow/serving:2.5.4-gpu"
else:
    DEPLOY_IMAGE = (
        f"{REGION}-docker.pkg.dev/"
        + PROJECT_ID
        + f"/{PRIVATE_REPO}"
        + "/tf_serving:cpu"
    )
    TF_IMAGE = "tensorflow/serving:2.5.4"

if not IS_COLAB:
    if DEPLOY_GPU:
        ! sudo docker pull tensorflow/serving:2.5.4-gpu
    else:
        ! sudo docker pull tensorflow/serving:2.5.4

    ! docker tag $TF_IMAGE $DEPLOY_IMAGE
    ! docker push $DEPLOY_IMAGE
else:
    # install docker daemon
    ! apt-get -qq install docker.io

print("Deployment:", DEPLOY_IMAGE, DEPLOY_GPU, DEPLOY_NGPU)

在Colab中运行

In [ ]:
%%bash -s $IS_COLAB $DEPLOY_IMAGE $TF_IMAGE
if [ $1 == "False" ]; then
  exit 0
fi
set -x
dockerd -b none --iptables=0 -l warn &
for i in $(seq 5); do [ ! -S "/var/run/docker.sock" ] && sleep 2 || break; done
docker pull $3
docker tag tensorflow/serving $2
docker push $2
kill $(jobs -p)

## Vertex AI模型监控简介

Vertex AI模型监控支持AutoML表格模型和自定义表格模型。您可以监控入站预测请求中特征的偏差和漂移检测，或者监测出站预测响应中特征归因（可解释AI）的偏差和漂移 -- 即，归因对输出（预测）的贡献分布。

以下是启用模型监控的基本步骤：

1. 将`Vertex AI`的AutoML或自定义表格模型部署到`Vertex AI Endpoint`。
2. 配置模型监控规范。
3. 将模型监控规范上传到`Vertex AI Endpoint`。
4. 上传或自动生成用于解析的`输入架构`。
5. 对于特征偏差检测，上传用于自动生成特征分布的训练数据。
6. 对于特征归因，上传相应的`Vertex AI可解释性`规范。

配置完成后，您可以启用/禁用监控，更改警报并更新模型监控配置。

启用模型监控时，采样的传入预测请求将被记录到一个BigQuery表中。然后将分析记录请求中包含的输入特征值，以便按指定的间隔基准进行偏差或漂移的分析。您可以设置取样率来监控生产输入到模型的子集，以及监控间隔。

模型监控服务需要知道如何解析特征值，这称为输入架构。对于AutoML表格模型，输入架构会自动生成。对于自定义表格模型，服务会尝试从前1000个预测请求中自动生成输入架构。或者，您可以上传输入架构。

对于偏差检测，监控服务需要训练数据中数值统计分布的基线。对于AutoML表格模型，这是自动推导的。对于自定义表格模型，您需要上传训练数据到服务中，并让服务自动推导分布。

对于特征归因偏差和漂移检测，在自定义表格模型中需要启用您部署的模型的`Vertex AI可解释性`。对于AutoML模型，`Vertex AI可解释性`会自动启用。

了解更多关于[Vertex AI模型监控简介](https://cloud.google.com/vertex-ai/docs/model-monitoring/overview)。

### 复制 TensorFlow Serving 的模型文物

*注意:* 对于 TF Serving，MODEL_DIR 必须以数字结尾的子文件夹，例如，1。

In [ ]:
MODEL_ARTIFACT_URI = "gs://mco-mm/churn"
MODEL_DIR = BUCKET_URI + "/model/1"

! gsutil cp -r $MODEL_ARTIFACT_URI $MODEL_DIR

### 上传模型工件作为`Vertex AI Model`资源

首先，您可以使用`upload()`方法将预先训练好的自定义表格模型工件作为`Vertex AI Model`资源上传，使用以下参数：

- `display_name`：`Model`资源的可读性名称。
- `artifact_uri`：模型工件的云存储位置。
- `serving_container_image`：在模型部署到`Vertex AI`时使用的服务容器映像。
- `serving_container_command`：要启动的服务二进制（HTTP服务器）。
- `serving_container_args`：要传递给服务二进制的参数。对于TensorFlow Serving，必需的参数包括：
    - `--model_name`：要分配给模型的可读性名称。
    - `--model_base_name`：在容器中存储模型工件的位置。Vertex服务将变量$(AIP_STORAGE_URI)设置为服务在容器中安装模型工件的位置。
    - `--rest_api_port`：要发送基于REST的预测请求的端口。可以是8080或8501（TensorFlow Serving的默认值）。
    - `--port`：要发送基于gRPC的预测请求的端口。对于TensorFlow Serving，应该是8500。
- `serving_container_health_route`：服务定期ping以验证服务二进制是否正在运行的URL。对于TensorFlow Serving，这将是/v1/models/\<model_name\>。
- `serving_container_predict_route`：用于将基于REST的预测请求路由到服务的URL。对于TF Serving，这将是/v1/models/[model_name]:predict。
- `serving_container_ports`：HTTP服务器监听请求的端口列表。
- `sync`：是否等待进程完成，或立即返回（异步）。

上传模型到Vertex模型资源将返回一个长时间运行的操作，因为可能需要一些时间。

*注意:* 您在上传模型路径时省略结尾的数字子文件夹（例如，/1）。Vertex服务将上传包含模型工件子文件夹上面的父文件夹--这正是TensorFlow Serving二进制文件期望的内容。

*注意:* 当您将模型工件上传到`Vertex AI Model`资源时，您需要指定相应的部署容器映像。

In [ ]:
MODEL_NAME = "churn_" + UUID

model = aiplatform.Model.upload(
    display_name="churn_" + UUID,
    artifact_uri=MODEL_DIR[:-2],
    serving_container_image_uri=DEPLOY_IMAGE,
    serving_container_health_route="/v1/models/" + MODEL_NAME,
    serving_container_predict_route="/v1/models/" + MODEL_NAME + ":predict",
    serving_container_command=["/usr/bin/tensorflow_model_server"],
    serving_container_args=[
        "--model_name=" + MODEL_NAME,
        "--model_base_path=" + "$(AIP_STORAGE_URI)",
        "--rest_api_port=8080",
        "--port=8500",
        "--file_system_poll_wait_seconds=31540000",
    ],
    serving_container_ports=[8080],
    sync=True,
)

print(model)

### 部署 `Vertex AI 模型` 资源到 `Vertex AI 端点` 资源

接下来，您可以使用 `deploy()` 方法将您的 `Vertex AI 模型` 资源部署到一个 `Vertex AI 端点` 资源，使用以下参数：

- `deploy_model_display`：部署模型的人类可读名称。
- `machine_type`：每个 VM 节点实例的机器类型。
- `min_replica_count`：自动扩展所需提供的节点的最小数量。
- `max_replica_count`：自动扩展所需提供的节点的最大数量。
- `accelerator_type`：每个提供的节点的 GPU 加速器类型（如果有的话）。
- `accelerator_count`：每个提供的节点的 GPU 加速器数量（如果有的话）。

In [ ]:
MIN_NODES = 1
MAX_NODES = 1

if GPU:
    endpoint = model.deploy(
        deployed_model_display_name="churn_" + UUID,
        machine_type=DEPLOY_COMPUTE,
        min_replica_count=MIN_NODES,
        max_replica_count=MAX_NODES,
        accelerator_type=DEPLOY_GPU.name,
        accelerator_count=DEPLOY_NGPU,
    )
else:
    endpoint = model.deploy(
        deployed_model_display_name="churn_" + UUID,
        machine_type=DEPLOY_COMPUTE,
        min_replica_count=MIN_NODES,
        max_replica_count=MAX_NODES,
    )

配置监视作业

配置监视作业包括以下规范：

- `alert_config`：发送监视警报的电子邮件地址。
- `schedule_config`：分析预测的时间窗口。
- `logging_sampling_strategy`：采样预测请求的比率。
- `drift_config`：要监视的特征和漂移阈值。
- `skew_config`：要监视的特征和偏差阈值。

### 配置警报规范

首先，您可以使用以下设置配置`alerting_config`规范：

- `user_emails`：要发送警报的一个或多个电子邮件地址列表。
- `enable_logging`：将检测到的异常流式传输到 Cloud Logging。默认为False。

In [ ]:
# Create alerting configuration.
alerting_config = model_monitoring.EmailAlertConfig(
    user_emails=[USER_EMAIL], enable_logging=True
)

### 配置监控间隔规范

接下来，您需要使用以下设置配置`schedule_config`规范：

- `monitor_interval`：设置模型监控作业的调度间隔，单位为小时。最小时间间隔为1小时。

In [ ]:
# Monitoring Interval
MONITOR_INTERVAL = 1  # @param {type:"number"}

# Create schedule configuration
schedule_config = model_monitoring.ScheduleConfig(monitor_interval=MONITOR_INTERVAL)

### 配置采样规范

接下来，您可以使用以下设置配置`logging_sampling_strategy`规范：

- `sample_rate`：作为百分比（介于0和1之间）的速率，用于随机采样用于监控的预测请求。 选择的样本将被记录到一个BigQuery表中。

In [ ]:
# Sampling rate (optional, default=.8)
SAMPLE_RATE = 0.5  # @param {type:"number"}

# Create sampling configuration
logging_sampling_strategy = model_monitoring.RandomSampleConfig(sample_rate=SAMPLE_RATE)

### 配置漂移检测规范

接下来，您可以使用以下设置配置 `drift_config` 规范：

- `drift_thresholds`：一个键值对字典，其中键是要监控漂移的输入特征，值是检测阈值。如果未指定，默认的特征漂移阈值为0.3（30％）。

*注意：*启用漂移检测是可选的。

In [ ]:
DRIFT_THRESHOLD_VALUE = 0.05

DRIFT_THRESHOLDS = {
    "country": DRIFT_THRESHOLD_VALUE,
    "cnt_user_engagement": DRIFT_THRESHOLD_VALUE,
}

drift_config = model_monitoring.DriftDetectionConfig(drift_thresholds=DRIFT_THRESHOLDS)

### 配置倾斜检测规范

接下来，您需要使用以下设置配置 `skew_config` 规范：

- `data_source`：原始训练数据集的数据源。数据源的格式默认为 BigQuery 表。否则，必须将设置 `data_format` 设置为以下某个值。数据的位置必须是 Cloud Storage 位置。
  - `csv`： 
  - `jsonl`：
  - `tf-record`：
- `skew_thresholds`：一组键/值对的字典，其中键是要监视倾斜的输入特征。值是检测阈值。当未指定时，特征的默认倾斜阈值为 0.3（30%）。
- `target_field`：训练数据集的目标标签

*注意：* 启用倾斜检测是可选的。

In [ ]:
# URI to training dataset.
DATASET_BQ_URI = "bq://mco-mm.bqmlga4.train"  # @param {type:"string"}
# Prediction target column name in training dataset.
TARGET = "churned"

SKEW_THRESHOLD_VALUE = 0.5

SKEW_THRESHOLDS = {
    "country": SKEW_THRESHOLD_VALUE,
    "cnt_user_engagement": SKEW_THRESHOLD_VALUE,
}

skew_config = model_monitoring.SkewDetectionConfig(
    data_source=DATASET_BQ_URI, skew_thresholds=SKEW_THRESHOLDS, target_field=TARGET
)

### 组装目标规范

最后，您可以将目标规范`objective_config`与以下设置一起组装起来：

- `skew_detection_config`：（可选）偏斜检测配置的规范。
- `drift_detection_config`：（可选）漂移检测配置的规范。
- `explanation_config`：（可选）启用特征归因监视时用于解释的规范。

In [ ]:
objective_config = model_monitoring.ObjectiveConfig(
    skew_detection_config=skew_config,
    drift_detection_config=drift_config,
    explanation_config=None,
)

### 创建输入模式

监控服务需要知道模型的特征和数据类型的特征输入，这被称为“输入模式”。 “输入模式”可以是以下之一：
- 预加载到监控服务。
- 在收到头1000个预测实例后由监控服务自动生成。

在本教程中，您将预加载“输入模式”。

#### 创建预定义的输入模式

预定义的“输入模式”被指定为一个YAML文件。在本例中，您检索用于训练数据的BigQuery模式，其中包括特征名称和数据类型，以生成YAML规范。预定义的“输入模式”必须加载到Cloud存储位置。

了解更多关于[用于解析输入的自定义实例模式](https://cloud.google.com/vertex-ai/docs/model-monitoring/overview#custom-input-schemas)。

In [ ]:
# Get the BQ table

table = bigquery.TableReference.from_string(DATASET_BQ_URI[5:])
bq_table = bqclient.get_table(table)

yaml = """type: object
properties:
"""

schema = bq_table.schema
for feature in schema:
    if feature.name == TARGET:
        continue
    if feature.field_type == "STRING":
        f_type = "string"
    else:
        f_type = "integer"
    yaml += f"""  {feature.name}:
    type: {f_type}
"""

yaml += """required:
"""
for feature in schema:
    if feature.name == TARGET:
        continue
    yaml += f"""- {feature.name}
"""

print(yaml)

with open("schema.yaml", "w") as f:
    f.write(yaml)

! gsutil cp schema.yaml {BUCKET_URI}/schema.yaml

创建监控任务

您可以使用 `aiplatform.ModelDeploymentMonitoringJob.create()` 方法，根据您的监控规范创建一个监控任务，包括以下参数：

- `display_name`：监控任务的可读名称。
- `project`：项目 ID。
- `region`：地区。
- `endpoint`：要启用监控的 `Vertex AI Endpoint` 的完全限定资源名称。
- `logging_sampling_strategy`：采样配置的规范。
- `schedule_config`：调度配置的规范。
- `alert_config`：警报配置的规范。
- `objective_configs`：目标配置的规范。
- `analysis_instance_schema_uri`：包含 `input schema` 的 YAML 文件的位置。

In [ ]:
monitoring_job = aiplatform.ModelDeploymentMonitoringJob.create(
    display_name="churn_" + UUID,
    project=PROJECT_ID,
    location=REGION,
    endpoint=endpoint,
    logging_sampling_strategy=logging_sampling_strategy,
    schedule_config=schedule_config,
    alert_config=alerting_config,
    objective_configs=objective_config,
    analysis_instance_schema_uri=f"{BUCKET_URI}/schema.yaml",
)

print(monitoring_job)

#### 监控作业的电子邮件通知。

已向警报配置中的电子邮件地址发送电子邮件通知，通知模型监控作业现在已启用。

内容如下：

<blockquote>
你好，Vertex AI 客户，

您收到此邮件是因为您正在使用Vertex AI模型监控服务。
此邮件是为了通知您，我们已收到您的请求为以下列出的预测端点设置漂移或偏差检测。从现在开始，传入的预测请求将被抽样并记录以供分析。
原始请求和响应将从预测服务收集，并保存在 bq://[您的项目ID].model_deployment_monitoring_[端点ID].serving_predict。
</blockquote>

监视作业状态

在启动`Vertex AI模型监视`作业之后，直到计算出`偏斜分布基线`之前，作业将处于`PENDING`状态。监视服务将启动一个批处理作业，从训练数据生成分布基线。

一旦基线分布生成，监视作业将进入`OFFLINE`状态。按照每个间隔的基础，比如每小时一次，监视作业将进入`RUNNING`状态，同时分析采样数据。完成后，它将返回到`OFFLINE`状态，等待下一次计划的分析。

In [ ]:
jobs = monitoring_job.list(filter=f"display_name=churn_{UUID}")
job = jobs[0]
print(job.state)

### 基线分布的自动生成

接下来，监控服务将创建一个批处理作业，对训练数据进行分析以生成基线分布。一旦完成，监控服务将开始按照指定的间隔进行监控。

In [ ]:
import time

# Pause a bit for the baseline distribution to be calculated
if os.getenv("IS_TESTING"):
    time.sleep(300)

### 生成用于偏斜检测的合成预测请求

接下来，您从BigQuery训练表中提取前1000个实例用于预测请求。您修改数据（合成数据）以触发从训练分布到服务分布的预测请求中的偏斜检测，具体操作如下：

- `country`：将所有值设为加拿大

In [ ]:
# Download the table.
table = bigquery.TableReference.from_string(DATASET_BQ_URI[5:])

rows = bqclient.list_rows(table, max_results=1000)

instances = []
for row in rows:
    instance = {}
    for key, value in row.items():
        if key == TARGET:
            continue
        if value is None:
            value = ""
        if key == "country":
            value = "Canada"
        instance[key] = value
    instances.append(instance)

print(len(instances))

### 发送预测请求

接下来，使用`predict()`方法将1000个预测请求发送到您的`Vertex AI Endpoint`资源。

In [ ]:
for instance in instances:
    response = endpoint.predict(instances=[instance])

prediction = response[0]

# print the prediction for the first instance
print(prediction[0])

### 记录抽样请求

一旦监控服务启动，抽样预测请求将被记录到云存储中。在接下来的监控间隔中，抽样预测将被复制到BigQuery日志表中。一旦条目在BigQuery表中，监控服务将分析抽样数据。

接下来，您需要等待第一个记录的条目出现在用于记录预测样本的BigQuery表中。由于您发送了1000个预测请求，抽样率为50%，您应该看到大约500条记录。

In [ ]:
while True:
    time.sleep(180)

    ENDPOINT_ID = endpoint.resource_name.split("/")[-1]

    table = bigquery.TableReference.from_string(
        f"{PROJECT_ID}.model_deployment_monitoring_{ENDPOINT_ID}.serving_predict"
    )
    rows = bqclient.list_rows(table)
    print(rows.total_rows)
    if rows.total_rows > 0:
        break

### 监控中的偏斜检测

下一个监控间隔将发生特征输入偏斜检测。在本教程中，您将监控间隔设置为一小时。因此，大约一个小时后，您的监控作业将从“离线”状态变为“运行”状态。在运行时，它将分析在此间隔期间从预测中记录的抽样表，并将它们与基线分布进行比较。

分析完成后，监控作业将通过电子邮件通知检测到的偏斜，本例中为“国家”，并且监控作业将进入“离线”状态，直到下一个间隔。

#### 等待监控间隔

从监控间隔进行分析到收到电子邮件警报可能需要40分钟左右。

内容会如下显示：

<blockquote>
   尊敬的 Vertex AI 客户，

您正在接收此邮件是因为您订阅了 Vertex AI 模型监控服务。
此邮件仅用于通知您，已检测到您部署的模型中存在一些异常情况，可能需要您的注意。

基本信息：

终端节点名称：projects/[your-project-id]/locations/us-central1/endpoints/3315907167046860800
监控作业：projects/[your-project-id]/locations/us-central1/modelDeploymentMonitoringJobs/8672170640054157312
统计和异常根路径（Google Cloud 存储）：gs://cloud-ai-platform-773884b1-2a32-48d6-8b83-c03cde416b68/model_monitoring/job-8672170640054157312
BigQuery 命令：SELECT * FROM `bq://[your-project-id].model_deployment_monitoring_3315907167046860800.serving_predict`

训练预测偏斜异常（原始特征）：

异常报告路径（Google Cloud 存储）：gs://cloud-ai-platform-773884b1-2a32-48d6-8b83-c03cde416b68/model_monitoring/job-8672170640054157312/serving/2022-08-25T00:00/stats_and_anomalies/<deployed-model-id>/anomalies/training_prediction_skew_anomalies

有关警报的更多信息，请访问模型监控警报页面。

部署模型 ID: <deployed-model-id>

特征名称	异常简要描述	异常长描述
country	训练和服务之间的 Linfty 距离高	训练和服务之间的 Linfty 距离为 0.947563（至少保留六个有效数字），高于阈值 0.5。具有最大差异值的特征值为：Canada
<blockquote>

In [ ]:
if os.getenv("IS_TESTING"):
    time.sleep(60 * 45)

生成用于漂移检测的合成预测请求

接下来，您从BigQuery训练表中提取相同的前1000个实例用于预测请求。您修改数据（合成数据）以触发预测请求中的漂移检测，训练分布与服务分布如下：

- `cnt_user_engagement`：增加4倍的值。

In [ ]:
# Download the table.
table = bigquery.TableReference.from_string(DATASET_BQ_URI[5:])

rows = bqclient.list_rows(table, max_results=1000)

instances = []
for row in rows:
    instance = {}
    for key, value in row.items():
        if key == TARGET:
            continue
        if value is None:
            value = ""
        elif key == "cnt_user_engagement":
            value = int(value * 4)
        instance[key] = value
    instances.append(instance)

print(len(instances))

生成预测请求

接下来，您可以使用`predict()`方法向您的`Vertex AI Endpoint`资源发送1000个预测请求。

In [ ]:
for instance in instances:
    response = endpoint.predict(instances=[instance])

prediction = response[0]

# print the prediction for the first instance
print(prediction[0])

记录抽样请求

在下一次监控间隔中，抽样预测结果将被复制到BigQuery日志表中。一旦条目进入BigQuery表中，监控服务将分析抽样数据。

接下来，您等待第一个记录条目出现在用于记录预测样本的BigQuery表中。由于您发送了1000个预测请求，采样率为50%，您应该会看到大约1000条记录。

In [ ]:
while True:
    time.sleep(180)

    ENDPOINT_ID = endpoint.resource_name.split("/")[-1]

    table = bigquery.TableReference.from_string(
        f"{PROJECT_ID}.model_deployment_monitoring_{ENDPOINT_ID}.serving_predict"
    )
    rows = bqclient.list_rows(table)
    print(rows.total_rows)
    if rows.total_rows > 550:
        break

### 监控过程中的漂移检测

特征输入漂移检测将在下一个监控间隔期间发生。在本教程中，您将监控间隔设置为一小时。因此，大约一个小时后，您的监控作业将从“离线”状态变为“运行”状态。在运行时，它将分析此间隔内预测中记录的采样表，并与先前监控间隔的分布进行比较。

分析完成后，监控作业将通过电子邮件通知检测到的漂移，即“cnt_user_engagement”，并且监控作业将进入“离线”状态，直到下一个间隔。

#### 等待监控间隔

从监控间隔上的分析开始到收到电子邮件警报可能需要40分钟甚至更多时间。

In [ ]:
if os.getenv("IS_TESTING"):
    time.sleep(60 * 45)

### 删除监控任务

您可以使用 `delete()` 方法删除监控任务。

In [ ]:
monitoring_job.pause()
monitoring_job.delete()

取消部署并删除“Vertex AI Endpoint”资源

您可以使用`delete()`方法删除您的“Vertex AI Endpoint”资源。在删除之前，必须先取消部署到您的“Vertex AI Endpoint”资源的任何模型。

In [ ]:
endpoint.undeploy_all()
endpoint.delete()

清理

要清理此项目中使用的所有谷歌云资源，您可以删除用于本教程的[谷歌云项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的各个资源。

In [ ]:
delete_bucket = False

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -rf {BUCKET_URI}

! rm -f schema.yaml

! bq rm -f {PROJECT_ID}.model_deployment_monitoring_{ENDPOINT_ID}